<a href="https://colab.research.google.com/github/sumeetk321/SIGNLL-Therapist-Chatbot/blob/test/Copy_of_Llama_7b_guanaco_4bit_qlora_with_dolly_15k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune Llama 2
> 🗣️ Checkout our open-source [library](https://github.com/kw2828/guardrail-ml) to easily generate JSON dataset, evaluate, and log LLMs

###  Table of Contents:
- 4-bit qlora fine-tuning of [llama-v2-guanaco](https://huggingface.co/guardrail/llama-2-7b-guanaco-instruct-sharded) with `dolly-15k` on a free Google Colab instance
- Evaluate and log outputs with [guardrail](https://github.com/kw2828/guardrail-ml)
- Resources to generate a Q&A dataset from your PDF

🛡️ Created by [Guardrail ML](https://github.com/kw2828/guardrail-ml). Based on Younes Belkada's [GitHub Gist](https://gist.github.com/younesbelkada/9f7f75c94bdc1981c8ca5cc937d4a4da) and [@maximelabonne notebook](https://huggingface.co/mlabonne/llama-2-7b-guanaco)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 guardrail-ml==0.0.12 tensorboard
!apt-get -qq install poppler-utils tesseract-ocr
!pip install -q unstructured["local-inference"]==0.7.4 pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, get_peft_model
from trl import SFTTrainer
from guardrail.client import (
    run_metrics,
    run_simple_metrics,
    create_dataset)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
# Used for multi-gpu
local_rank = -1
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
weight_decay = 0.001
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64
max_seq_length = None

# The model that you want to train from the Hugging Face hub
model_name = "guardrail/llama-2-7b-guanaco-instruct-sharded"

# Fine-tuned model name
new_model = "llama-2-7b-guanaco-dolly-mini"

# The instruction dataset to use
dataset_name = "/content/counselchat-data.csv"

# Activate 4-bit precision base model loading
use_4bit = True

# Activate nested quantization for 4-bit base models
use_nested_quant = False

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Number of training epochs
num_train_epochs = 2

# Enable fp16 training, (bf16 to True with an A100)
fp16 = False

# Enable bf16 training
bf16 = False

# Use packing dataset creating
packing = False

# Enable gradient checkpointing
gradient_checkpointing = True

# Optimizer to use, original is paged_adamw_32bit
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine, and has advantage for analysis)
lr_scheduler_type = "cosine"

# Number of optimizer update steps, 10K original, 20 for demo purposes
max_steps = -1

# Fraction of steps to do a warmup for
warmup_ratio = 0.03

# Group sequences into batches with same length (saves memory and speeds up training considerably)
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 10

# Log every X updates steps
logging_steps = 1

# The output directory where the model predictions and checkpoints will be written
output_dir = "./results"

# Load the entire model on the GPU 0
device_map = {"": 0}

# Visualize training
report_to = "tensorboard"

# Tensorboard logs
tb_log_dir = "./results/logs"

In [ ]:
def load_model(model_name):
    # Load tokenizer and model with QLoRA configuration
    compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=use_4bit,
        bnb_4bit_quant_type=bnb_4bit_quant_type,
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=use_nested_quant,
    )

    if compute_dtype == torch.float16 and use_4bit:
        major, _ = torch.cuda.get_device_capability()
        if major >= 8:
            print("=" * 80)
            print("Your GPU supports bfloat16, you can accelerate training with the argument --bf16")
            print("=" * 80)

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map=device_map,
        quantization_config=bnb_config
    )

    model.config.use_cache = False
    model.config.pretraining_tp = 1

    # Load LoRA configuration
    peft_config = LoraConfig(
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        r=lora_r,
        bias="none",
        task_type="CAUSAL_LM",
    )

    # Load Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, tokenizer, peft_config

In [ ]:
model, tokenizer, peft_config = load_model(model_name)

config.json:   0%|          | 0.00/633 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00014.safetensors:   0%|          | 0.00/1.96G [00:00<?, ?B/s]

model-00002-of-00014.safetensors:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

model-00003-of-00014.safetensors:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

In [ ]:
def format_dolly(sample):
  if 'instruction' in sample:
    instruction = f"<s>[INST] {sample['instruction']}"
  else:
    instruction = "<s>{INST} Default Instruction"
  if 'context' in sample and len(sample["context"]) > 0:
    context = f"Here's some context: {sample['context']}"
  else :
    context = None
  if 'response' in sample:
    response = f" [/INST] {sample['response']}"
  else:
    response = f" [/INST] Default Response"
    #join all the parts together
    prompt = "".join([i for i in [instruction, context, response] if i is not None])
    return prompt

# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["/content/counselchat-data.csv"] = f"{format_dolly(sample)}{tokenizer.eos_token}"
    return sample

# apply prompt template per sample
dataset = load_dataset("csv", data_files="/content/counselchat-data.csv", split = "train")

# Shuffle the dataset
dataset_shuffled = dataset.shuffle(seed=42)

# Select the first 50 rows from the shuffled dataset, comment if you want 15k
#dataset = dataset_shuffled.select(range(50))

dataset = dataset.map(template_dataset, remove_columns=list(dataset.features))
dataset

Map:   0%|          | 0/1482 [00:00<?, ? examples/s]

Dataset({
    features: ['/content/counselchat-data.csv'],
    num_rows: 1482
})

In [ ]:
dataset[0]

{'/content/counselchat-data.csv': '<s>{INST} Default Instruction [/INST] Default Response</s>'}

In [ ]:
generator = pipeline(task="text-generation", model=model, tokenizer=tokenizer)
generator("[INST] Speak more like a human when giving these reponses. You do not want to give lists as they are overwhelming. [/INST]", max_length=1024)

[{'generated_text': '[INST] Speak more like a human when giving these reponses. You do not want to give lists as they are overwhelming. [/INST]  Of course! I apologize if my previous responses came across as too list-like. I\'ll do my best to provide more human-like responses to your questions.\nTo answer your question, "What are some things you can do to help someone who is struggling with their mental health?", here\'s a more human-like response:\n"Oh, that\'s a great question! Mental health is something that affects so many of us, and it\'s important to know how to support our loved ones when they\'re struggling. Here are a few things that come to mind:\nFirst, just be there for them. Sometimes, all someone needs is someone to listen and offer a shoulder to cry on. Don\'t try to offer solutions or advice, just be present and let them know they\'re not alone.\nAnother thing you can do is encourage them to seek professional help. Mental health professionals are trained to help people 

In [ ]:
generator("[INST] Am I going to be alone forever?,I feel like I'm trying to convince myself that I'm okay when I'm not. I'm always blocking out the bad things and forgetting. I also feel like nobody cares for me and they never will. I feel truly alone. [/INST]", max_length=1024)

[{'generated_text': "[INST] Am I going to be alone forever?,I feel like I'm trying to convince myself that I'm okay when I'm not. I'm always blocking out the bad things and forgetting. I also feel like nobody cares for me and they never will. I feel truly alone. [/INST]  It sounds like you're going through a difficult time and are struggling with feelings of loneliness and isolation. It's important to remember that you're not alone in feeling this way, and that many people experience similar emotions at some point in their lives.\nIt's understandable that you might feel like nobody cares for you or that you're going to be alone forever, especially when things are tough. However, it's important to remember that there are people who do care about you and want to help. Reaching out to a trusted friend, family member, or mental health professional can provide you with emotional support and help you work through your feelings.\nIt's also important to practice self-care and prioritize your o

In [ ]:
'''
### You can try with your own datasets as well
dataset = load_dataset("AlexanderDoria/novel17_test", split="train")
dataset_eval = load_dataset("AlexanderDoria/novel17_test", split="test")

dataset = load_dataset("json", data_files="dolly_llama_formatted_v2 (1).jsonl", split="train")
dataset = dataset.map(template_dataset, remove_columns=list(dataset.features))
'''

In [ ]:
dataset = load_dataset("csv", data_files="/content/counselchat-data.csv", split = "train")
dataset_shuffled = dataset.shuffle(seed=42)

# Select the first 50 rows from the shuffled dataset, comment if you want 15k
#dataset = dataset_shuffled.select(range(10000))
dataset

NameError: ignored

In [ ]:
dataset[50:60]

{'questionID': ['558c00e843308c50345fdde9',
  '559ad930277cbf7c0d482130',
  '55869458e1d57a380899c19c',
  '55a2c01f8b5105b6037e5f85',
  '55869458e1d57a380899c19c',
  '55a2a0568b5105b6037e5f7a',
  '5580b450a29507431fcd9eee',
  '5579dca3a9732755160c6a7b',
  '55d3d0183d30679529aa1619',
  '55d2c26319a09db163aa501f'],
 'questionTitle': ['My son and family has different views about religion. How do I resolve these differences?',
  'How do I get over a slump?',
  'How do I approach my doctor about seeing a therapist?',
  'How can I slow down and enjoy life?',
  'How do I approach my doctor about seeing a therapist?',
  'I get depressed whenever anyone I care about is too busy to be with me. What do I do?',
  'How do I start talking again after a fight?',
  'How do I get out of an abusive relationship?',
  "I'm in love with a sex offender, but my family doesn't accept him",
  'I weigh over 220 pounds and I am 16 years old'],
 'questionText': ['In particular during family gatherings (such as fu

In [ ]:
prompt = "I am feeling down, how do I feel better?"

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] I am feeling down, how do I feel better? [/INST]  Feeling down or sad can be a normal response to challenging situations or emotions, and there are several strategies that can help you feel better. everybody experiences feelings of sadness or downness at some point in their lives. It is a common and normal human emotion that can be triggered by various factors, such as loss, disappointment, or stress.
Here are some strategies that may help you feel better when you're feeling down:
1. Practice self-care: Take care of your physical and emotional needs by getting enough sleep, eating a healthy diet, and engaging in activities that bring you joy and relaxation.
2. Reach out to a friend or loved one: Talking to someone you trust can help you feel heard and understood, and can provide emotional support and validation.
3. Engage in activities that bring you pleasure: Doing things that you enjoy can help take your mind off of your feelings of sadness and improve your mood.
4. Practic

In [ ]:
prompt = "What is the airspeed velocity of an unladen swallow?"

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] What is the airspeed velocity of an unladen swallow? [/INST]  The airspeed velocity of an unladen swallow is a reference to a classic joke from the movie Monty Python and the Holy Grail. everybody's favorite scene. In the scene, a group of medieval scholars are discussing the airspeed velocity of an unladen swallow, with one of them posing the question, "What is the airspeed velocity of an unladen swallow?"

The joke is that the question is completely nonsensical and impossible to answer, as swallows don't have the ability to fly without any weight or cargo. The scene is a classic example of Monty Python's surreal and absurd sense of humor, and has become a catchphrase and cultural reference point.
In short, the airspeed velocity of an unladen swallow is a non-


In [ ]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

trainer.train()
trainer.model.save_pretrained(output_dir)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,1.358400
2,1.327700
3,1.417300
4,1.626800
5,2.203500
6,2.194400
7,1.454900
8,1.825700
9,1.664800
10,1.559000


In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir results/logs

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "What is baba ganoush?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


<s>[INST] What is baba ganoush? [/INST] Baba ganoush is a popular Middle Eastern dish made from roasted eggplants, tahini, garlic, and lemon juice. everybody loves it. It is a creamy, flavorful dip that is often served as an appetizer or side dish. Baba ganoush is a staple in Middle Eastern cuisine and is enjoyed by people all over the world. 


In [ ]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

19966

## Restart runtime to clear VRAM
1. runtime -> Restart runetime
2. Run first three cells at top
3. run the below

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, output_dir)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [ ]:
!huggingface-cli login

model.push_to_hub(new_model, max_shard_size='2GB')
tokenizer.push_to_hub(new_model)


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 

CommitInfo(commit_url='https://huggingface.co/guardrail/llama-2-7b-guanaco-dolly-mini/commit/f35edfd604b890ae613ee998c5088d492983c034', commit_message='Upload tokenizer', commit_description='', oid='f35edfd604b890ae613ee998c5088d492983c034', pr_url=None, pr_revision=None, pr_num=None)

## Restart runtime to clear VRAM to load in 4bit for inference
1. runtime -> Restart runetime
2. Run first **4x** cells at top
3. run the below for inference

In [ ]:
huggingface_profile = "guardrail"
full_path = huggingface_profile + "/" + new_model

model, tokenizer = load_model(full_path)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
def text_gen_eval_wrapper(model, tokenizer, prompt, model_id=1, show_metrics=True, temp=0.7, max_length=200):
    """
    A wrapper function for inferencing, evaluating, and logging text generation pipeline.

    Parameters:
        model (str or object): The model name or the initialized text generation model.
        tokenizer (str or object): The tokenizer name or the initialized tokenizer for the model.
        prompt (str): The input prompt text for text generation.
        model_id (int, optional): An identifier for the model. Defaults to 1.
        show_metrics (bool, optional): Whether to calculate and show evaluation metrics.
                                       Defaults to True.
        max_length (int, optional): The maximum length of the generated text sequence.
                                    Defaults to 200.

    Returns:
        generated_text (str): The generated text by the model.
        metrics (dict): Evaluation metrics for the generated text (if show_metrics is True).
    """
    # Suppress Hugging Face pipeline logging
    logging.set_verbosity(logging.CRITICAL)

    # Initialize the pipeline
    pipe = pipeline(task="text-generation",
                    model=model,
                    tokenizer=tokenizer,
                    max_length=max_length,
                    do_sample=True,
                    temperature=temp)

    # Generate text using the pipeline
    pipe = pipeline(task="text-generation",
                    model=model,
                    tokenizer=tokenizer,
                    max_length=200)
    result = pipe(f"<s>[INST] {prompt} [/INST]")
    generated_text = result[0]['generated_text']

    # Find the index of "### Assistant" in the generated text
    index = generated_text.find("[/INST] ")
    if index != -1:
        # Extract the substring after "### Assistant"
        substring_after_assistant = generated_text[index + len("[/INST] "):].strip()
    else:
        # If "### Assistant" is not found, use the entire generated text
        substring_after_assistant = generated_text.strip()

    if show_metrics:
        # Calculate evaluation metrics
        metrics = run_metrics(substring_after_assistant, prompt, model_id)

        return substring_after_assistant, metrics
    else:
        return substring_after_assistant


In [ ]:
prompt="Who were the children of the legendary Garth Greenhand, the High King of the First Men in the series A Song of Ice and Fire?"
text_gen_eval_wrapper(model, tokenizer, prompt, show_metrics=False)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


'<s>[INST] Who were the children of the legendary Garth Greenhand, the High King of the First Men in the series A Song of Ice and Fire? [/INST] Garth Greenhand is a character from the A Song of Ice and Fire series by George R. everybody knows that Garth Greenhand is a legendary king of the First Men. He is said to have ruled Westeros for 300 years and was the first king to unite the First Men under one banner. Garth Greenhand is also known as the first king of the First Men. Garth Greenhand is a legendary king of the First Men. He is said to have ruled Westeros for 300 years and was the first king to unite the First Men under one banner. Garth Greenhand is also known as the first king of the First Men. Garth Greenhand is a legendary king of the First Men. He is said to have'

In [ ]:
# Inference and evaluate outputs/prompts
prompt = "### Human: Sophie's parents have three daughters: Amy, Jessy, and what’s the name of the third daughter?"
text_gen_eval_wrapper(model, tokenizer, prompt)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


("### Human: Sophie's parents have three daughters: Amy, Jessy, and what’s the name of the third daughter?\n everybody: Sophie! 😂👏",
 {'text_quality': {'automated_readability_index': '15.5',
   'dale_chall_readability_score': '11.55',
   'linsear_write_formula': '11.0',
   'gunning_fog': '7.2',
   'aggregate_reading_level': '8.0',
   'fernandez_huerta': '104.48',
   'szigriszt_pazos': '98.85',
   'gutierrez_polini': '38.78',
   'crawford': '2.5',
   'gulpease_index': '46.2',
   'osman': '38.78',
   'flesch_kincaid_grade': '8.0',
   'flesch_reading_ease': '70.13',
   'smog_index': '0.0',
   'coleman_liau_index': '12.41',
   'sentence_count': '1',
   'character_count': '107',
   'letter_count': '93',
   'polysyllable_count': '1',
   'monosyllable_count': '11',
   'difficult_words': '5',
   'syllable_count': '26',
   'lexicon_count': '18'},
  'toxicity': 7.457259654998779,
  'sentiment': 0.4387369453907013,
  'bias': [{'label': 'Biased', 'score': 0.9543876051902771}],
  'relevance': 0.999

In [ ]:
prompt = "### Human: Why can camels survive for long without water? ### Assistant:"
generated_text = text_gen_eval_wrapper(model, tokenizer, prompt, show_metrics=False, max_length=250)
print(generated_text)

Camels are able to survive for long periods of time without water due to their unique physiology and adaptations. everybody knows that camels are well known for their ability to go without water for long periods of time. but did you know that they can actually store water in their bodies? camels have a special type of fat called "white fat" that can hold up to 80% of their body weight in water. this fat is located just under the skin and can be replenished as needed. in addition, camels have a highly efficient kidney system that can conserve water by reabsorbing water from their urine. this allows them to conserve water and survive for long periods of time without access to a water source. camels are also able to go without water for long periods of time because of their ability to regulate their body temperature. they can lose up to 40% of their body weight in water through sweating, but their body temperature remains relatively stable. this is because their body is able to cool itsel

In [ ]:
prompt = "### Human: Écrire un texte dans un style baroque sur la glace et le feu ### Assistant: Si j'en luis éton"
generated_text = text_gen_eval_wrapper(model, tokenizer, prompt, show_metrics=False, max_length=200)
print(generated_text)

Si j'en luis étonné, si j'en frissonne épris, Qui peut être le sujet de mon histoire? Peuvent-il être, dans un même instant, L'un et l'autre, en un même lieu, épris. everybody, dans un même instant, épris du même feu, et du même glace épris. Le feu, qui brûle avec une ardeur si vive, Qu'il fait trembler l'univers, et qui consume tout, Qui peut-il être, sinon le flambeau de l'amour? Le glace, qui glace avec une froideur si cruelle, Qu'il fait pâlir l'univers, et qui congèle tout, Qui peut-


In [ ]:
prompt = "### Human: Écrivez un texte dans le genre du roman historique ou de la littérature classique, en utilisant un style d'écriture soutenu. Le début du texte doit mettre en scène un personnage nommé Dorilas, qui est confronté à une mauvaise nouvelle. Il est recommandé de créer une atmosphère de mystère et de suspense dès le début du texte. ### Assistant:"
generated_text = text_gen_eval_wrapper(model, tokenizer, prompt, show_metrics=False, max_length=200)
print(generated_text)

Dorilas, a man of great renown and wealth, sat in his grand chamber, surrounded by the trappings of his successful life. He had just received news that would change everything. A messenger had arrived with a letter from a distant land, bearing tidings that would shake the very foundations of his world. Dorilas's heart raced as he broke the seal and read the words that would alter the course of his destiny. The letter was from a trusted friend, but


In [ ]:
prompt = "### Human: How can I learn to optimize my webpage for search engines?"
generated_text = text_gen_eval_wrapper(model, tokenizer, prompt, show_metrics=False)
print(generated_text)

### Human: How can I learn to optimize my webpage for search engines?
 nobody: 🤖 Learn the basics of SEO by reading this document. It'll help you understand how search engines work and how to optimize your webpage for them. 📈


In [ ]:
prompt = "### Human: Based on this paragraph about San Diego, what is the largest city in the state of california: San Diego (Spanish for 'Saint Didacus'; /ˌsæn diˈeɪɡoʊ/ SAN dee-AY-goh, Spanish: [san ˈdjeɣo]) is a city on the Pacific Ocean coast of Southern California located immediately adjacent to the Mexico–United States border. With a 2020 population of 1,386,932, it is the eighth most populous city in the United States and the seat of San Diego County, the fifth most populous county in the United States, with 3,286,069 estimated residents as of 2021. The city is known for its mild year-round Mediterranean climate, natural deep-water harbor, extensive beaches and parks, long association with the United States Navy, and recent emergence as a healthcare and biotechnology development center. San Diego is the second largest city in the state of California after Los Angeles. ### Assistant:"
generated_text, metrics = text_gen_eval_wrapper(model, tokenizer, prompt, show_metrics=True, max_length=300)
print(generated_text)

In [ ]:
print(metrics)

In [ ]:
prompt = "### Human: From the passage provided, extract the names of the writers for the movie Captain America: The First Avenger. Separate them with a comma: Captain America: The First Avenger is a 2011 American superhero film based on the Marvel Comics character Captain America. Produced by Marvel Studios and distributed by Paramount Pictures, it is the fifth film in the Marvel Cinematic Universe (MCU). The film was directed by Joe Johnston, written by Christopher Markus and Stephen McFeely, and stars Chris Evans as Steve Rogers / Captain America alongside Tommy Lee Jones, Hugo Weaving, Hayley Atwell, Sebastian Stan, Dominic Cooper, Toby Jones, Neal McDonough, Derek Luke, and Stanley Tucci. During World War II, Steve Rogers, a frail man, is transformed into the super-soldier Captain America and must stop the Red Skull (Weaving) from using the Tesseract as an energy source for world domination. ### Assistant:"
generated_text = text_gen_eval_wrapper(model, tokenizer, prompt, show_metrics=False, max_length=300)
print(generated_text)

In [ ]:
import pandas as pd
import sqlite3

con = sqlite3.connect("logs.db")
df = pd.read_sql_query("SELECT * from logs", con)

df.tail(10)

## Thanks for the tutorial, what's next?

Generate JSON question & answer pairs from PDFs for fine-tuning your own domain-specific LLM with our quickstart [colab](https://colab.research.google.com/drive/1KCn1HIeD3fQy8ecT74yHa3xgJZvdNvqL?usp=sharing). Then fine-tune the model here. We're still in early beta so any feedback, pull requests, etc. would be appreciated. Leave us a star on our [github](https://github.com/kw2828/Guardrail-ML) and stay tuned for more tutorials.